<a href="https://colab.research.google.com/github/noa-blenkitny/IR_Project/blob/main/backend_index_creation_and_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-dd36  GCE       4                                       RUNNING  us-central1-a


In [4]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [5]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import numpy as np
import builtins
from contextlib import closing
import hashlib
from time import time #we use it for testing

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  6 17:24 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [8]:
bucket_name = '3b1u8c3k6e4t122' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
postings_gcp/anchor_index0_000.bin
postings_gcp/anchor_index0_001.bin
postings_gcp/anchor_index0_002.bin
postings_gcp/anchor_index0_003.bin
postings_gcp/anchor_index0_posting_locs.pickle
postings_gcp/anchor_index100_000.bin
postings_gcp/anchor_index100_001.bin
postings_gcp/anchor_index100_002.bin
postings_gcp/anchor_index100_003.bin
postings_gcp/anchor_index100_004.bin
postings_gcp/anchor_index100_posting_locs.pickle
postings_gcp/anchor_index101_000.bin
postings_gcp/anchor_index101_001.bin
postings_gcp/anchor_index101_002.bin
postings_gcp/anchor_index101_003.bin
postings_gcp/anchor_index101_posting_locs.pickle
postings_gcp/anchor_index102_000.bin
postings_gcp/anchor_index102_001.bin
postings_gcp/anchor_index102_002.bin
postings_gcp/anchor_index102_003.bin
postings_gcp/anchor_index102_posting_locs.pickle
postings_gcp/anchor_index103_000.bin
postings_gcp/anchor_index103_001.bin
postings_gcp/anchor_index103_002.bin
postings_gcp/anchor_index103_003.bin
postings_gcp/anchor_in

In [9]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
parquetFile.count()

6348910

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex
from inverted_index_gcp import MultiFileReader
from inverted_index_gcp import MultiFileWriter


In [13]:
#rdds of which we build the different indexes on
body_data = parquetFile.select("id","text").rdd
title_data = parquetFile.select("id","title").rdd
anchor_data = parquetFile.select("id","anchor_text").rdd

**creating {docid:title} dict**



In [12]:
titles_dict= title_data.collectAsMap()
with open('titles_dict.pkl', 'wb') as f: #in gcp: need to change it 
  pickle.dump(titles_dict, f)
print("done creating title dict")

done creating title dict


**helper func to help during creation of indexes**

In [14]:
#using code from assignment 3- gcp
#funcs to help build the inverted index
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def calc_doc_len(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  
  return len(tokens)

def word_count(text, id):
  
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords ]
  word_count_list = []
  words_dict= OrderedDict()
  for word in tokens:
    words_dict[word] = words_dict.get(word,0) + 1
  for item in words_dict.items():
    word_count_list.append((item[0],(id,item[1])))
  return word_count_list

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return  sorted(unsorted_pl,key=lambda y: y[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''

  return postings.mapValues(lambda x : len(x))

def partition_postings_and_write(postings,new_index_name):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  #partitioning RDD to (bucket_id, [(w0, posting_list_0)) entries 
  postings = postings.map(lambda x: (new_index_name+str(token2bucket_id(x[0])),x))
  #reducing it to (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...])
  postings = postings.groupByKey()
  #writing each bucket to disk
  #need to change it in gcp to:
  postings = postings.map(lambda y: (InvertedIndex.write_a_posting_list(y,'3b1u8c3k6e4t122')))
  return postings
  
def calc_tf_idf(tf, N , df, doc_len):
  tf_idf = (tf/doc_len)* math.log(N/df)
  return tf_idf


**body index**

In [15]:

def create_inverted_index_with_tf_idf_weights(data,new_index_name):
    # word counts map - posting lists
    word_counts = data.flatMap(lambda x: word_count(x[1], x[0]))
    #united posting lists
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    #Next, we will filter out rare words, words that appear in 10 or fewer documents (when working on the entire corpus, we will increase this threshold to a minimum of 50 documents).
    #in gcp: change filter to 50
    postings = postings.filter(lambda x: len(x[1])>50) 
    # RDD to map {doc_id,[(token,tf),(token,tf)....]}
    tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    # groupBy doc_id
    tf = tf.groupByKey().mapValues(list)
    #RDD to map doc_id to doc_len
    doc_lens = tf.map(lambda x: (x[0],builtins.sum([tup[1] for tup in x[1]])))
    # turn the doc_lens RDD into dict - TODO: check how its gonna be for the whole corpus
    doc_lens = doc_lens.collectAsMap()
    # filtering postings and calculate df
    #in gcp:
    
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()

    # partition posting lists and write out- in gcp
    #write posting lists to disk and returns posting locs dict for each bucket
    #_ = partition_postings_and_write(postings_filtered,new_index_name).collect()

    #in colab:
    #write posting lists to disk and returns posting locs dict for each bucket
    posting_locs_list = partition_postings_and_write(postings,new_index_name).collect()
    

    # collect all posting lists locations into one super-set- gcp code:
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
      if not blob.name.endswith("pickle"):
        continue
      elif blob.name.startswith(f'postings_gcp/{new_index_name}'):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
          super_posting_locs[k].extend(v)

    # merge the posting locations into a single dict- good for local test in colab
#     super_posting_locs = defaultdict(list)
#     for posting_loc in posting_locs_list:
#         for k, v in posting_loc.items():
#           super_posting_locs[k].extend(v)

    new_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    new_index.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    new_index.df = w2df_dict
    # the number of the docs in the corpus - using to calc TF-IDF
    new_index.N= data.count()
    # print(new_index.N)
    # tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    tf_idf = tf.flatMap(lambda x: [(x[0],(tup[0], calc_tf_idf(tup[1], new_index.N , new_index.df[tup[0]], doc_lens[x[0]]))) for tup in x[1]] ) 
    tf_idf = tf_idf.groupByKey().mapValues(dict)
    #truning tf_idf rdd to docid: (doc_len,sum-mechane of cosSim)
    tf_idf = tf_idf.map(lambda x: (x[0],(doc_lens[x[0]],builtins.sum(math.pow(y,2) for y in x[1].values() ))))   
    new_index.weights = tf_idf.collectAsMap()
    # write the global stats out
    new_index.write_index('.', new_index_name)

    return new_index

In [ ]:
body_index = create_inverted_index_with_tf_idf_weights(body_data,"body_index")
print("done creating body _index")

done creating body _index


**title index**

In [17]:
def create_inverted_index_with_binary_weights(data,new_index_name):
    # word counts map - posting lists
    word_counts = data.flatMap(lambda x: word_count(x[1], x[0]))
    #united posting lists
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    # RDD to map {doc_id,[(token,tf),(token,tf)....]}
    tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    # groupBy doc_id
    tf = tf.groupByKey().mapValues(list)
   
    #calculate df   
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()


    #in colab:
    #write posting lists to disk and returns posting locs dict for each bucket
    posting_locs_list = partition_postings_and_write(postings,new_index_name).collect()
    

    # collect all posting lists locations into one super-set- gcp code:
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
      if not blob.name.endswith("pickle"):
        continue
      elif blob.name.startswith(f'postings_gcp/{new_index_name}'):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
          super_posting_locs[k].extend(v)

    # merge the posting locations into a single dict- good for local test in colab
#     super_posting_locs = defaultdict(list)
#     for posting_loc in posting_locs_list:
#         for k, v in posting_loc.items():
#           super_posting_locs[k].extend(v)

    new_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    new_index.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    new_index.df = w2df_dict
    # the number of the docs in the corpus - using to calc TF-IDF
    new_index.N= data.count()
    # write the global stats out
    new_index.write_index('.', new_index_name)

    return new_index

In [18]:
title_index = create_inverted_index_with_binary_weights(title_data,"title_index")
print("done creating title index")

done creating title index


22/01/06 19:21:22 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_16_python !
22/01/06 19:21:22 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_17_python !


In [33]:
from contextlib import closing
#posting list by word- could be useful when doing calculation dynamically in relation to the words in the query
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [34]:
title = InvertedIndex.read_index('.', "title_index")
print(read_posting_list(title,"python"))

[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0),

**anchor index**

In [19]:
def create_inverted_index_with_binary_weights_anchor(data,new_index_name):
    #rdd of (destid,text),(destid,text)...
    data= data.flatMap(lambda l: [(value[0], value[1]) for value in l[1]])  
    data= data.groupByKey().mapValues(list)
    data=data.mapValues(lambda x: ','.join(set(item for item in x)))
    # word counts map - posting lists
    word_counts = data.flatMap(lambda x: word_count(x[1], x[0]))
    #print(word_counts.take(1))
    #united posting lists
    #word_counts = word_counts.map(lambda x: x[1])
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    #print(postings.lookup("python"))
    # RDD to map {doc_id,[(token,tf),(token,tf)....]}
    tf = postings.flatMap(lambda x: [(tup[0], (x[0],tup[1])) for tup in x[1]] ) 
    # groupBy doc_id
    tf = tf.groupByKey().mapValues(list)
   
    #calculate df
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()
    # partition posting lists and write out- in gcp
    #_ = partition_postings_and_write(postings_filtered).collect()

    #in colab:
    #write posting lists to disk and returns posting locs dict for each bucket
    posting_locs_list = partition_postings_and_write(postings,new_index_name).collect()
    

    # collect all posting lists locations into one super-set- gcp code:
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
      if not blob.name.endswith("pickle"):
        continue
      elif blob.name.startswith(f'postings_gcp/{new_index_name}'):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
          super_posting_locs[k].extend(v)

    # merge the posting locations into a single dict- good for local test in colab
#     super_posting_locs = defaultdict(list)
#     for posting_loc in posting_locs_list:
#         for k, v in posting_loc.items():
#           super_posting_locs[k].extend(v)

    new_index = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    new_index.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    new_index.df = w2df_dict
    # the number of the docs in the corpus - using to calc TF-IDF
    new_index.N= data.count()
    # write the global stats out
    new_index.write_index('.', new_index_name)

    return new_index

In [21]:
anchor_index = create_inverted_index_with_binary_weights_anchor(anchor_data,"anchor_index")
print("done creating anchor index")

done creating anchor index


**page rank**

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  #rdd of (srcid,[destid1,destid2...])
  key_edges = pages.mapValues(lambda x: set(item[0] for item in x))
  #rdd of (srcid,destid1),(srcid,destid2)...
  edges=key_edges.flatMap(lambda l: [(l[0], value) for value in l[1]])
  vertices = edges.keys().union(edges.values()).distinct().map(lambda x: [x])
  return edges, vertices

def write_page_rank_to_file():
  #in gcp:
  #pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd ##this is anchor_data but on all corpus
  # construct the graph 
  #edges, vertices = generate_graph(pages_links)
  #in colab:
  edges, vertices = generate_graph(anchor_data)
  # compute PageRank
  edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
  verticesDF = vertices.toDF(['id']).repartition(124, 'id')
  g = GraphFrame(verticesDF, edgesDF)
  pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
  pr = pr_results.vertices.select("id", "pagerank")
  #pr = pr.sort(col('pagerank').desc()) #i think we dont need to sort it
  pr = pr.rdd
  pr = pr.map(lambda x : (x[0],x[1]))
  pr_dict = pr.collectAsMap()
  with open('pr.pkl', 'wb') as f: #in gcp: need to change it 
    pickle.dump(pr_dict, f)
    
write_page_rank_to_file()
print("done creating page rank dict")
#reading page rank:
#with open('pr.pkl', 'rb') as f:
  #pr = pickle.loads(f.read())

done creating page rank dict


In [16]:
with open('pr.pkl', 'rb') as f:
  pr = pickle.loads(f.read())
print(pr[50991931])

0.45572746486411153


**page views- we already calculated it for the whole corpus**

In [ ]:
# # Paths
# # Using user page views (as opposed to spiders and automated traffic) for the 
# # month of August 2021
# pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
# p = Path(pv_path) 
# pv_name = p.name
# pv_temp = f'{p.stem}-4dedup.txt'
# pv_clean = f'{p.stem}.pkl'
# # Download the file (2.3GB) 
# !wget -N $pv_path
# # Filter for English pages, and keep just two fields: article ID (3) and monthly 
# # total number of page views (5). Then, remove lines with article id or page 
# # view values that are not a sequence of digits.
# !bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# # Create a Counter (dictionary) that sums up the pages views for the same 
# # article, resulting in a mapping from article id to total page views.
# wid2pv = Counter()
# with open(pv_temp, 'rt') as f:
#   for line in f:
#     parts = line.split(' ')
#     wid2pv.update({int(parts[0]): int(parts[1])})
# # write out the counter as binary file (pickle it)
# with open(pv_clean, 'wb') as f:
#   pickle.dump(wid2pv, f)
# # read in the counter
# # with open(pv_clean, 'rb') as f:
# #   wid2pv = pickle.loads(f.read())

**reading the page views**

In [ ]:
#read in the counter
with open(pv_clean, 'rb') as f:
  wid2pv = pickle.loads(f.read())

**Metrices for report**

In [ ]:
############################need to remember to change all builtins.round to round in the final version########
def intersection(l1,l2):      
    """
    This function perform an intersection between two lists.

    Parameters
    ----------
    l1: list of documents. Each element is a doc_id.
    l2: list of documents. Each element is a doc_id.

    Returns:
    ----------
    list with the intersection (without duplicates) of l1 and l2
    """
    return list(set(l1)&set(l2))

#remember to copy the avarege percsision from run _frontend_in_colab
def average_precision(true_list, predicted_list, k=40):
    #true_set = frozenset(true_list)
    true_set = builtins.frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    #for i,doc_id in enumerate(predicted_list):        
    for i,doc_id in builtins.enumerate(predicted_list):
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)           
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    #return round(sum(precisions)/len(precisions),3)
    return builtins.round(builtins.sum(precisions)/len(precisions),3)


def precision_at_k(true_list,predicted_list,k=40):    
    """
    This function calculate the precision@k metric.

    Parameters
    -----------
    true_list: list of relevant documents. Each element is a doc_id.
    predicted_list: sorted list of documents predicted as relevant. Each element is a doc_id. Sorted is performed by relevance score
    k: integer, a number to slice the length of the predicted_list
    
    Returns:
    -----------
    float, precision@k with 3 digits after the decimal point.
    """      
    if (k > len(predicted_list)):
        k = len(predicted_list)
    denominator = k
    if k == 0:
      return 0
    numerator = len(intersection(predicted_list[:k],true_list))
    #return round(numerator / denominator,3)
    return builtins.round(numerator / denominator,3)

def recall_at_k(true_list,predicted_list,k=40):
    """
    This function calculate the recall@k metric.

    Parameters
    -----------
    true_list: list of relevant documents. Each element is a doc_id.
    predicted_list: sorted list of documents predicted as relevant. Each element is a doc_id. Sorted is performed by relevance score
    k: integer, a number to slice the length of the predicted_list
    
    Returns:
    -----------
    float, recall@k with 3 digits after the decimal point.
    """      
    if (k < len(predicted_list)):
        predicted_list = predicted_list[:k]
    denominator = len(true_list)
    if (denominator  == 0):
      return 0.0
    numerator = len(intersection(predicted_list,true_list))
    #return round(numerator / denominator,3)
    return builtins.round(numerator / denominator,3)

def f_score(true_list,predicted_list,k=40):
    """
    This function calculate the f_score@k metric.

    Parameters
    -----------
    true_list: list of relevant documents. Each element is a doc_id.
    predicted_list: sorted list of documents predicted as relevant. Each element is a doc_id. Sorted is performed by relevance score
    k: integer, a number to slice the length of the predicted_list
    
    Returns:
    -----------
    float, f-score@k with 3 digits after the decimal point.
    """   
    P =  precision_at_k(true_list,predicted_list,k)
    R = recall_at_k(true_list,predicted_list,k)
    if ( P == 0 or R == 0):
      return 0.000
    denominator = P + R
    numerator = 2 * P * R 
    #return round(numerator / denominator,3)
    return builtins.round(numerator / denominator,3)


###need to change it to our terms
def evaluate(true_relevancy,predicted_relevancy,k,print_scores=True):
    """
    This function calculates multiple metrics and returns a dictionary with metrics scores across different queries.
    Parameters
    -----------
    true_relevancy: list of tuples indicating the relevancy score for a query. Each element corresponds to a query.
    Example of a single element in the list: turn it to [(query as string,true list),()]
                                            (3, {'question': ' what problems of heat conduction in composite slabs have been solved so far . ',
                                            'relevance_assessments': [(5), (6), (90), (91), (119), (144), (181), (399), (485)]})
     
    predicted_relevancy: a dictionary of the list. Each key represents the query_id. The value of the dictionary is a sorted list of relevant documents and their scores.
                         The list is sorted by the score.  
    Example:
            key: 1
            value: [(13, 17.256625), (486, 13.539465), (12, 9.957595), (746, 9.599499999999999), (51, 9.171265), .....]            
            
    k: integer, a number to slice the length of the predicted_list
    
    print_scores: boolean, enable/disable a print of the mean value of each metric.
    
    Returns:
    -----------
    a dictionary of metrics scores as follows: 
                                                        key: metric name
                                                        value: list of metric scores. Each element corresponds to a given query.
    """    
    recall_lst = []
    precision_lst = []   
    avg_precision_lst = []
    f_score_lst = []
    metrices = {'recall@k':recall_lst,
                'precision@k':precision_lst,
                'f_score@k': f_score_lst,
                'MAP@k':avg_precision_lst}
    
    for query_id, query_info in true_relevancy:
        predicted = predicted_relevancy[query_id]
        ground_true = [int(doc_id) for doc_id in query_info['relevance_assessments']]
    
        recall_lst.append(recall_at_k(ground_true,predicted,k=k))
        precision_lst.append(precision_at_k(ground_true,predicted,k=k))
        f_score_lst.append(f_score(ground_true,predicted,k=k))
        avg_precision_lst.append(average_precision(ground_true,predicted,k=k))

    if print_scores:
        for name,values in metrices.items():
                print(name,sum(values)/len(values))

    return metrices    

**tests**

In [ ]:
########right now we can't test anchor index- we'll test it on the whole corpus in gcp#######
#need to import search_backend in order to evaluate (we use search functions)#
#time to create index:
res=[]
#body:
t_start_body = time()
new_body_index = create_inverted_index_with_tf_idf_weights(body_data,"new_body_index")
body_index_const_time = time() - t_start_body
res.append(("body index creation time",body_index_const_time))
#title:
t_start_title = time()
new_title_index = create_inverted_index_with_binary_weights(title_data,"new_title_index")
title_index_const_time = time() - t_start_title
res.append(("title index creation time",title_index_const_time))

# #anchor:
# t_start_anchor = time()
# new_anchor_index = create_inverted_index_with_binary_weights_anchor(anchor_data,"new_anchor_index")
# anchor_index_const_time = time() - t_start_anchor
# res.append(("anchor index creation time",anchor_index_const_time))

#time to search in index:
###one word query####
#body:
t_search_body = time()
body_search_result = backend_search_body("python",new_body_index)
body_search_result_time = time() - t_search_body
res.append(("one word query body search time",body_search_result_time))

#title:
t_search_title = time()
title_search_result = backend_search_title_anchor("python",new_title_index)
title_search_result_time = time() - t_search_title
res.append(("one word query title search time",title_search_result_time))

# #anchor:
# t_search_anchor = time()
# anchor_search_result = backend_search_title_anchor("python",new_anchor_index)
# anchor_search_result_time = time() - t_search_anchor
# res.append(("one word query anchor search time",anchor_search_result_time))

###few word query####
#body:
t_search_body = time()
few_body_search_result = backend_search_body("why+do+men+have+nipples",new_body_index)
few_body_search_result_time = time() - t_search_body
res.append(("few word query body search time",few_body_search_result_time))

#title:
t_search_title = time()
few_title_search_result = backend_search_title_anchor("why+do+men+have+nipples",new_title_index)
few_title_search_result_time = time() - t_search_title
res.append(("few word query title search time",few_title_search_result_time))

# #anchor:
# t_search_anchor = time()
# few_anchor_search_result = backend_search_title_anchor("why+do+men+have+nipples",new_anchor_index)
# few_anchor_search_result_time = time() - t_search_anchor
# res.append(("few word query anchor search time",few_anchor_search_result_time))

#evaluation from hw4:
body_search_result= [x[0] for x in body_search_result]
title_search_result= [x[0] for x in title_search_result]
#anchor_search_result= [x[0] for x in anchor_search_result]
few_body_search_result= [x[0] for x in few_body_search_result]
few_title_search_result= [x[0] for x in few_title_search_result]
#few_anchor_search_result= [x[0] for x in few_anchor_search_result]

#precision
res.append(("precision@40, one word search, body index",precision_at_k(query_python_true_list,body_search_result)))
res.append(("precision@40, one word search, title index",precision_at_k(query_python_true_list,title_search_result)))
#res.append(("precision@40, one word search, anchor index",precision_at_k(query_python_true_list,anchor_search_result)))
res.append(("precision@40, few word search, body index",precision_at_k(query_nipples_true_list,few_body_search_result)))
res.append(("precision@40, few word search, title index",precision_at_k(query_nipples_true_list,few_title_search_result)))
#res.append(("precision@40, few word search, anchor index",precision_at_k(query_nipples_true_list,few_anchor_search_result)))
#recall
res.append(("recall@40, one word search, body index",recall_at_k(query_python_true_list,body_search_result)))
res.append(("recall@40, one word search, title index",recall_at_k(query_python_true_list,title_search_result)))
#res.append(("recall@40, one word search, anchor index",recall_at_k(query_python_true_list,anchor_search_result)))
res.append(("recall@40, few word search, body index",recall_at_k(query_nipples_true_list,few_body_search_result)))
res.append(("recall@40, few word search, title index",recall_at_k(query_nipples_true_list,few_title_search_result)))
#res.append(("recall@40, few word search, anchor index",recall_at_k(query_nipples_true_list,few_anchor_search_result)))
#f_score
res.append(("f_score@40, one word search, body index",f_score(query_python_true_list,body_search_result)))
res.append(("f_score@40, one word search, title index",f_score(query_python_true_list,title_search_result)))
#res.append(("f_score@40, one word search, anchor index",f_score(query_python_true_list,anchor_search_result)))
res.append(("f_score@40, few word search, body index",f_score(query_nipples_true_list,few_body_search_result)))
res.append(("f_score@40, few word search, title index",f_score(query_nipples_true_list,few_title_search_result)))
#res.append(("f_score@40, few word search, anchor index",f_score(query_nipples_true_list,few_anchor_search_result)))
#map@k
res.append(("map@40, one word search, body index",average_precision(query_python_true_list,body_search_result)))
res.append(("map@40, one word search, title index",average_precision(query_python_true_list,title_search_result)))
#res.append(("map@40, one word search, anchor index",average_precision(query_python_true_list,anchor_search_result)))
res.append(("map@40, few word search, body index",average_precision(query_nipples_true_list,few_body_search_result)))
res.append(("map@40, few word search, title index",average_precision(query_nipples_true_list,few_title_search_result)))
#res.append(("map@40, few word search, anchor index",average_precision(query_nipples_true_list,few_anchor_search_result)))

print(res)

In [ ]:
import matplotlib.pyplot as plt

def plot_metric_with_differnt_k_values(true_relevancy,predicted_relevancy,metrices_names,k_values):    
    """
    This function plot a for each given metric its value depands on k_values as line chart.
    This function does not return any value.

    Parameters
    ----------
    true_relevancy: list of tuples indicating the relevancy score for a query. Each element corresponds to a query.
    Example of a single element in the list: 
                                            (3, {'question': ' what problems of heat conduction in composite slabs have been solved so far . ',
                                            'relevance_assessments': [(5, 3), (6, 3), (90, 3), (91, 3), (119, 3), (144, 3), (181, 3), (399, 3), (485, 1)]})

    predicted_relevancy: a dictionary of the list. Each key represents the query_id. The value of the dictionary is a sorted list of relevant documents and their scores.
                         The list is sorted by the score.  
    Example:
            key: 1
            value: [(13, 17.256625), (486, 13.539465), (12, 9.957595), (746, 9.599499999999999), (51, 9.171265), .....]

    metrices_names: list of string representing the metrices to plot. For example: ['precision@k','recall@k','f_score@k']

    k_values: list of integer of different k values. For example [1,3,5]

    returns: 
    plot values in format : 
    
    statistics[metric_name] = [values , k_values]

    """
    statistics = {}
    temp_stats = {}
    for k in k_values:
      ranks = evaluate(true_relevancy,predicted_relevancy,k,print_scores=False)
      for metric_name in metrices_names:
        if metric_name in temp_stats:
          temp_stats.get(metric_name).append((np.mean(ranks[metric_name]),k))

        else:
          temp_stats[metric_name] = [(np.mean(ranks[metric_name]),k)]


    for metric_name in metrices_names:
      values = [x[0] for x in  temp_stats[metric_name]]
      statistics[metric_name] = [values,k_values]
      plt.plot(k_values,values,label=metric_name)
      plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
      plt.title("metrics values for anchor index")
      plt.xlabel('k values')
      plt.ylabel('metric values')
      #plt.show()

    return statistics


def plot_metric_different_quieries(true_relevancy,predicted_relevancy,metrices_names,k):
    """
    This function plot for each given metric its value across all queries.
    This function does not return any value.

    Parameters
    ----------
    true_relevancy: list of tuples indicating the relevancy score for a query. Each element corresponds to a query.
    Example of a single element in the list: 
                                            (3, {'question': ' what problems of heat conduction in composite slabs have been solved so far . ',
                                            'relevance_assessments': [(5, 3), (6, 3), (90, 3), (91, 3), (119, 3), (144, 3), (181, 3), (399, 3), (485, 1)]})

    predicted_relevancy: a dictionary of the list. Each key represents the query_id. The value of the dictionary is a sorted list of relevant documents and their scores.
                         The list is sorted by the score.  
    Example:
            key: 1
            value: [(13, 17.256625), (486, 13.539465), (12, 9.957595), (746, 9.599499999999999), (51, 9.171265), .....]

    metrices_names: list of string representing the metrices to plot. For example: ['precision@k','recall@k','f_score@k']
    k: integer, a number to slice the length of the predicted_list.
    
    """
    ranks = evaluate(true_relevancy,predicted_relevancy,k,print_scores=False)
    x = [query[0] for query in true_relevancy]
    for metric_name in metrices_names:
      values = ranks[metric_name]
      index = metrices_names.index(metric_name)
      metric_name_for_plot= ['precision@10','recall@10','f_score@10','MAP@10'] ####remmeber to change to different k when needed (probably 40)
      plt.plot(x,values,marker= "o",label=metric_name_for_plot[index])
      plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
      plt.title("different queries for body index")
      plt.xlabel('query id')
      plt.ylabel('values')
      #plt.show()

**code for testing and printing plots**

In [ ]:
true_relevancy = [(1,{'question': 'python','relevance_assessments' : query_python_true_list}),(2,{'question': 'why do men have nipples','relevance_assessments' : query_nipples_true_list})]
predicted_relevancy_for_body = {1:body_search_result,2:few_body_search_result}
#print(predicted_relevancy_for_body)
predicted_relevancy_for_title = {1:title_search_result,2:few_title_search_result}
#predicted_relevancy_for_anchor = {1:anchor_search_result,2:few_anchor_search_result}
#metrices_names = ['precision@k','recall@k','f_score@k','MAP@k']
metrices_names = ['precision@k','recall@k','f_score@k','MAP@k']
#k_values = [1,3,5,10,20,30,40]
k_values = [1,3,5,10]
plot_metric_with_differnt_k_values(true_relevancy,predicted_relevancy_for_body,metrices_names,k_values)
plot_metric_different_quieries(true_relevancy,predicted_relevancy_for_body,metrices_names,10)